<a href="https://colab.research.google.com/github/Bunny825/men_formal_informal_densenet121/blob/main/men's_formal_vs_informal_densenet121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest',
    channel_shift_range=20.0
)


test_datagen = ImageDataGenerator(rescale=1.0/255)


train_generator=train_datagen.flow_from_directory("/content/drive/MyDrive/men_formal_informal_data/train",target_size=(224,224),batch_size=32,class_mode='binary')
test_generator=test_datagen.flow_from_directory("/content/drive/MyDrive/men_formal_informal_data/test",target_size=(224,224),batch_size=32,class_mode='binary')

Found 640 images belonging to 2 classes.
Found 74 images belonging to 2 classes.


In [46]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Flatten
from tensorflow.keras.optimizers import RMSprop

densenet = DenseNet121(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
for layer in densenet.layers:
  layer.trainable=False

for layer in densenet.layers[-10:]:
  layer.trainable = True

model=Sequential()
model.add(densenet)
model.add(GlobalAveragePooling2D())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)             │ (None, 7, 7, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_12          │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 32)                  │          32,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,070,337 (26.97 MB)

 Trainable params: 200,961 (785.00 KB)

 Non-trainable params: 6,869,376 (26.20 MB)

In [47]:


model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20,
)



Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.5690 - loss: 0.7192 - val_accuracy: 0.5946 - val_loss: 0.6631
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 650ms/step - accuracy: 0.6151 - loss: 0.6340 - val_accuracy: 0.6622 - val_loss: 0.6315
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 640ms/step - accuracy: 0.6822 - loss: 0.5979 - val_accuracy: 0.7027 - val_loss: 0.6085
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 647ms/step - accuracy: 0.7471 - loss: 0.5552 - val_accuracy: 0.7432 - val_loss: 0.5784
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 635ms/step - accuracy: 0.7339 - loss: 0.5476 - val_accuracy: 0.7432 - val_loss: 0.5548
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 622ms/step - accuracy: 0.7594 - loss: 0.5141 - val_accuracy: 0.7703 - val_loss: 0.5385
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 621ms/step - accuracy: 0.7721 - loss: 0.4954 - val_accuracy: 0.7162 - val_loss: 0.5423
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 616ms/step - accuracy: 0.7747 - loss: 0.4835 - val_accurac

In [48]:
model.save("men_formal_informal_model_densenet121_82%_acc.h5")  # Saves in .h5 format
